In [2]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("chinm1010r/results")
model = AutoModelForSeq2SeqLM.from_pretrained("chinm1010r/results")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

In [7]:
dataset = load_dataset("xsum")

# Take a subset of the dataset
train_subset = dataset['train'].shuffle(seed=42).select(range(10000))
val_subset = dataset['validation'].select(range(1000))  # Smaller validation set

def preprocess_data(batch):
    inputs = ["summarize: " + doc for doc in batch["document"]]
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        batch["summary"], max_length=128, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = train_subset.map(preprocess_data, batched=True, remove_columns=["document", "summary", "id"])
val_data = val_subset.map(preprocess_data, batched=True, remove_columns=["document", "summary", "id"])


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-small")  # You can use "t5-base" or "t5-large" as well
tokenizer = T5Tokenizer.from_pretrained("t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import TrainerCallback

class SummaryGenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, model, eval_dataset):
        self.tokenizer = tokenizer
        self.model = model
        self.eval_dataset = eval_dataset

    def on_epoch_end(self, args, state, control, **kwargs):
        # Generate summaries from the evaluation dataset
        for example in self.eval_dataset:
            # Assuming 'input_ids' and 'attention_mask' are present after preprocessing
            input_ids = example["input_ids"]
            # Decode the input IDs to get the original text
            input_text = self.tokenizer.decode(input_ids, skip_special_tokens=True)
            inputs = self.tokenizer("summarize: " + input_text, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
            summary_ids = self.model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
            summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            print(f"Input: {input_text}\nSummary: {summary}\n")


# Initialize the callback
summary_callback = SummaryGenerationCallback(tokenizer, model, val_data)


In [12]:
print(val_data[0])  # Check the first example


{'input_ids': [21603, 10, 37, 1215, 18, 19915, 53, 3, 13720, 11958, 24283, 3415, 3991, 3, 8321, 12, 8, 264, 26, 1924, 5716, 2941, 3, 18, 3, 9, 7813, 12, 3033, 540, 21, 7904, 29, 2600, 5, 1363, 264, 26, 1924, 6, 6862, 6, 19, 22801, 4977, 28, 17813, 10740, 262, 89, 15, 6, 8537, 6, 17557, 6, 943, 11, 7872, 6, 6426, 5, 2276, 2741, 53, 44, 8, 3525, 24042, 2283, 6, 66, 662, 11958, 8, 326, 1433, 5, 37, 1567, 3, 21679, 12, 10883, 2319, 84, 3, 18280, 808, 286, 344, 2628, 11, 5832, 3084, 6, 45, 14599, 6, 262, 89, 15, 11, 17557, 6, 13, 15993, 9145, 6, 11, 7872, 6, 45, 493, 226, 1306, 6, 33, 788, 12, 1518, 3689, 16, 1718, 5, 328, 130, 66, 1883, 30, 15794, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    callbacks=[SummaryGenerationCallback(tokenizer, model, val_data)],
)

trainer.train()


Step,Training Loss,Validation Loss
500,0.625300,0.611829
1000,0.612300,0.608643
1500,0.632400,0.607013
2000,0.634700,0.604301
2500,0.641400,0.603198
3000,0.662700,0.600612
3500,0.627300,0.599585
4000,0.650300,0.597755
4500,0.644700,0.597546
5000,0.628400,0.596892


Streaming output truncated to the last 5000 lines.
Summary: A shoplifter has been jailed for aggravated vehicle-taking, sending indecent letters and burglary, a court heard.

Input: summarize: Haki Africa said the figure could be higher but some families were reluctant to come forward for fear of victimisation by state security agents. The police oversight body said 52 officers were on trial in connection with such killings countrywide. The coastal region has suffered several terror attacks in recent years. The Somali Islamist militant al-Shabab group has been targeting Kenya over the deployment of Kenyan forces to Somalia. Most of the victims were youths including returnees from Somalia who had surrendered after the government promised them amnesty, Haki Africa's report said. Others were allegedly killed as a result of excessive use of force during police crackdowns on protests and other religious groups that had been accused of radicalising young people. Saada Suleiman, who attended 

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Input: summarize: Two generals said the jihadist group had suffered the losses as troops and allied fighters, backed by US-led air strikes, advanced on several axes. Up to 5,000 IS fighters were believed to be in Mosul ahead of the assault. Despite the territorial gains, commanders have warned that securing Mosul could take weeks, if not months. About 50,000 Iraqi security forces personnel, Kurdish Peshmerga fighters, Sunni Arab tribesmen and Shia militiamen are involved in the operation. More than 100 US military personnel are embedded with them, advising commanders and helping direct coalition air strikes. Other US troops are providing fire support from nearby bases. Lt Gen Stephen Townsend, the commander of US forces in Iraq, said on Wednesday that the coalition forces had delivered more than 2,100 aerial bombs, artillery and mortar shells, rockets and missiles since 17 October. "This relentless campaign of strikes has removed hundreds of fighters, weapons, and key leaders from the 

TrainOutput(global_step=7500, training_loss=0.6337202107747396, metrics={'train_runtime': 3100.0293, 'train_samples_per_second': 9.677, 'train_steps_per_second': 2.419, 'total_flos': 4060254044160000.0, 'train_loss': 0.6337202107747396, 'epoch': 3.0})

In [15]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.5951952338218689, 'eval_runtime': 15.6446, 'eval_samples_per_second': 63.92, 'eval_steps_per_second': 15.98, 'epoch': 3.0}


In [16]:
model.save_pretrained("./t5_xsum_final")
tokenizer.save_pretrained("./t5_xsum_final")


('./t5_xsum_final/tokenizer_config.json',
 './t5_xsum_final/special_tokens_map.json',
 './t5_xsum_final/spiece.model',
 './t5_xsum_final/added_tokens.json')

In [19]:
import os

print(os.getcwd())

/content


In [21]:
print(model) # Will print a summary of the model architecture
print(model.config) # Will print the model's configuration details

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop